# Build your own Application
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://&lt;board_ip_address&gt;/lab for this notebook.
</div>

This notebook shows your how to build your custom composable application from the ground up

## Aims
* Create self contained application


## Table of Contents
* [Introduction](#intro)
* [Download Composable Overlay](#download)
* [Start HDMI Video](#start_hdmi)
* [Compose Pipeline](#compose)
* [Build the Application](#build)
* [Conclusion](#conclusion)

----

## Revision History

* v1.0 | 30 March 2021 | First notebook revision.
* v1.1 | 11 August 2021 | Update notebook to composable overlay API 1.0.0

----

## Introduction <a class="anchor" id="intro"></a>

We are going to recreate the [CornerDetect application](../applications/02_corner_detect_app.ipynb) using the intermediate API and expose some of the runtime configuration parameters using [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)

## Download Composable Overlay <a class="anchor" id="download"></a>

Start by downloading the composable overlay onto the FPGA

In [ ]:
from pynq import Overlay
from pynq.lib.video import *
from composable_pipeline import *

ol = Overlay("../overlay/cv_dfx_4_pr.bit")

cpipe = ol.composable

## Start HDMI Video <a class="anchor" id="start_hdmi"></a>

Get `HDMIVideo` object and start video

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to connect HDMI cables to a valid video source and screen may cause the notebook to hang
</div>

In [ ]:
video = HDMIVideo(ol)
video.start()

## Compose Pipeline <a class="anchor" id="compose"></a>

First we need to load the partial bitstreams to bring `fast` and `cornerHarris` functionality

In [ ]:
cpipe.loadIP(['pr_0/fast_accel', 'pr_1/cornerHarris_accel'])

Grab handlers, compose the pipeline and visualize it

In [ ]:
rgb2gray = cpipe.rgb2gray_accel
gray2rgb = cpipe.gray2rgb_accel
fast = cpipe.pr_0.fast_accel
harr = cpipe.pr_1.cornerHarris_accel

cpipe.compose([cpipe.hdmi_in_in, rgb2gray , fast, gray2rgb, cpipe.hdmi_in_out])

cpipe.graph

## Build the Application <a class="anchor" id="build"></a>

In the following cells we will define some useful functions to help us change the functionality of the application

Declare the `threshold` and `K` values as [IntSlider](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#IntSlider) and [FloatSlider](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#FloatSlider) respectively 

In [ ]:
from ipywidgets import widgets, IntSlider, FloatSlider, interact

thr = IntSlider(min=0, max=255, step=1, value=20)
k_harris = FloatSlider(min=0, max=0.2, step=0.002, value=0.04, description='K')

Declare `swap` function that enables to change between `fast` and `cornerHarris`

In [ ]:
algorithm = 'Fast'
def swap():
    global algorithm
    global thr
    if algorithm == 'Fast':
        cpipe.replace((fast, harr))
        algorithm = 'Harris'
        thr.max = 1024
        thr.value = 422
    else:
        cpipe.replace((harr, fast))
        algorithm = 'Fast'
        thr.max = 255
        thr.value = 20

In [ ]:
def app(new_algorithm, threshold, k):
    global thr
    global k_harris
    if new_algorithm != algorithm:
        swap()
    elif new_algorithm == 'Fast':
        fast.threshold = threshold
        k_harris.disabled = True
    else:
        harr.threshold = threshold
        k_harris.disabled = False
        harr.k = k

## Run the Application <a class="anchor" id="run"></a>

Finally we can use [interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Using-Interact), which automatically creates user interface (UI) controls, to run our application. The first argument is the function we want to call and the following are the argument to such function.

In [ ]:
interact(app, new_algorithm=['Fast','Harris'], threshold=thr, k=k_harris);

## Stop HDMI Video <a class="anchor" id="stop_hdmi"></a>

Finally stop the HDMI video pipeline

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to stop the HDMI Video may hang the board 
when trying to download another bitstream onto the FPGA
</div>

In [ ]:
video.stop()

----

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has shown how to create an application from the ground up. However, it is recommended to inherit from `PipelineApp` to create your own application.

[⬅️ Using Dynamic IP on the Composable Pipeline](05_dynamic_ip.ipynb) | | [Advanced Features ➡️](07_advanced_features.ipynb)

Copyright &copy; 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----